In [22]:
#Imports
#Loading our data
import csv as csv
import cv2
import numpy as np

#Keras imports
from keras.models import Sequential
from keras.layers import Flatten, Dense

Using TensorFlow backend.


### Loading the data

In order to load our data we firstly read the driving_log.csv file which contains the paths to the left, center and right cameras and also info about steering angle, throttle, break and speed. 

We will want to automate the loading process in order to easily load our own data which is stored in different directories. For that we used several libraries such as csv (read csv files) and cv2 (read images).

In [19]:
#Reading the files

#Reading the csv file
def readCSV(in_path):
    lines = []
    with open(in_path) as csvfile:
        reader = csv.reader(csvfile)
        next(reader, None) #Skip header
        for line in reader:
            lines.append(line)
            
    return lines

def readData(in_dir, in_lines):
    images = []
    measurements = []
    for line in in_lines:
        source_path = line[0]
        filename = source_path.split('/')[-1]
        path = in_dir + filename
        image = cv2.imread(path)
        images.append(image)
        measurement = float(line[3])
        measurements.append(measurement)
    
    return np.array(images), np.array(measurements)

In [21]:
imgs_path = './data/IMG/'
csv_path = './data/driving_log.csv'

X_train, y_train = readData(imgs_path, readCSV(csv_path))

print('Data loaded successfully!')

Data loaded successfully!


### Basic neural network using Keras

We will implement a basic neural network in order to verify that everything is working before implementing a more complex model. This network will just going to be a flattened image connected to a single output node. This single output node will predict the stearing angle, thus converting this model into a regression network. In contrast with a classification network, we may apply a softmax activation function to the output layer. Nevertheless in this case we will not use an activation function. We will directly predict the steering measurement. 

For this basic implementation we will use Keras as a library which works with tensorflow as backend. This will simplify our implementation and will be great for prototyping. Let's go ahead!

In [ ]:
#Model definition

model = Sequential()
model.add(Flatten(input_shape=(160,320,3)))
model.add(Dense(1))

#Model compilation
#For the loss function we will use Mean Squared Error (MSE). We will minimize the 
#error between the steering measurement which the network predicts and the ground 
#truth steering measurements provided by the dataset
model.compile(loss='mse', optimizer='adam')
#we also shuffle the data and split off 20% of the data to use for a validation set
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=6)

#Keras by default will run 10 epochs. Nevertheless with 10 epochs we will 
#overfit the training data. For that reason we will only perform 6 epochs
model.save('basic_model.h5')

Train on 6428 samples, validate on 1608 samples
Epoch 1/6
2944/6428 [============>.................] - ETA: 1s - loss: 11397960.7675